In [1]:
from os import chdir
chdir('/Users/lananhnguyen/Desktop/thesis/thesis_code')
import main.packages.unchain_chain as chain
import main.packages.mine_generic as mine_g
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
import re
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [2]:
def take_y_reals():
    cats = ["food", "energy", "neig", "services"]
    
    concatenated_df = pd.DataFrame()

    for cat in cats:
        cat_df = mine_g.load_excel(f"data/hicp_cat_raw/prc_hicp_{cat}.xlsx", name = f"hicp_{cat}", subset=True, verbose=0)
        #cat_df = cat_df[(cat_df.index > mine_g.train_test_split_date)]
        concatenated_df = pd.concat([concatenated_df, cat_df], axis=1)
    return concatenated_df

y_real = take_y_reals()

In [3]:
import os
import pandas as pd

def concatenate_csv_files(folder_path):
    """
    Concatenates CSV files in a folder column-wise into a single DataFrame.

    Args:
    - folder_path (str): Path to the folder containing CSV files.

    Returns:
    - concatenated_df (DataFrame): Concatenated DataFrame.
    """

    csv_files = [file for file in os.listdir(folder_path) if file.endswith('.csv')]

    concatenated_df = pd.DataFrame()

    # Iterate through each CSV file
    for file in csv_files:
        file_path = os.path.join(folder_path, file)
        df = pd.read_csv(file_path)
        
        # Add each file's data as new columns in the concatenated DataFrame
        concatenated_df = pd.concat([concatenated_df, df], axis=1)
        

    return concatenated_df

# Example usage:
folder_path = "data/forecast_results"
forecast_yoy_all_cat = concatenate_csv_files(folder_path)



In [4]:
def extract_forecast_model_h(forecast_all_cat_df, model, h):
    """
    in 4 df, extract columns with model's name and horizon
    """

    pattern = rf"^{model}.*h_{h}"
    matching_cols = [col for col in forecast_all_cat_df.columns if re.search(pattern, col)]
    df_specific_model_h = forecast_all_cat_df.loc[:, matching_cols]
    return df_specific_model_h

model = 'prc'
h = 1

new_df = extract_forecast_model_h(forecast_yoy_all_cat, model, h)

date_range = pd.date_range(start=mine_g.train_test_split_date + pd.DateOffset(months=1), end=mine_g.max_X_date, freq='M')
new_df.set_index(date_range, inplace=True)

In [5]:
food_chained = chain.transform_back_chained(new_df.iloc[:, 0], y_real.iloc[:, 0])
food_unchain = chain.unchain_series(food_chained, y_real.iloc[:, 0])

In [6]:
food_chained

2016-01-31    100.704347
2016-02-29    100.649580
2016-03-31    100.758589
2016-04-30    101.499623
2016-05-31    101.680922
                 ...    
2022-08-31    127.943809
2022-09-30    129.602990
2022-10-31    131.606672
2022-11-30    133.345814
2022-12-31    134.989547
Freq: M, Length: 84, dtype: float64

In [7]:
energy_chained = chain.transform_back_chained(new_df.iloc[:, 1], y_real.iloc[:, 1])
energy_unchain = chain.unchain_series(energy_chained, y_real.iloc[:, 1])

In [8]:
neig_chained = chain.transform_back_chained(new_df.iloc[:, 2], y_real.iloc[:, 2])
neig_unchained = chain.unchain_series(neig_chained, y_real.iloc[:, 2])

In [9]:
neig_chained

2016-01-31     99.218510
2016-02-29    100.087865
2016-03-31    101.185392
2016-04-30    101.167958
2016-05-31    101.233596
                 ...    
2022-08-31    112.750789
2022-09-30    114.215532
2022-10-31    114.982121
2022-11-30    116.370062
2022-12-31    116.650774
Freq: M, Length: 84, dtype: float64

In [10]:
services_chained = chain.transform_back_chained(new_df.iloc[:, 3], y_real.iloc[:, 3])
services_unchained = chain.unchain_series(services_chained, y_real.iloc[:, 3])

In [11]:
weight_f = pd.read_excel('data/hicp_cat_raw/prc_hicp_weight_food.xlsx', sheet_name='Sheet 1', skiprows=7)

weight_f = weight_f.iloc[1, :].to_frame().reset_index()
weight_f.columns = weight_f.iloc[0]
weight_f = weight_f[1:]
weight_f.rename(columns={"TIME": "date", "Germany": 'weight_f'}, inplace=True)

/Users/lananhnguyen/anaconda3/envs/thesis/lib/python3.11/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [12]:
weight_e = pd.read_excel('data/hicp_cat_raw/prc_hicp_weight_energy.xlsx', sheet_name='Sheet 1', skiprows=7)

weight_e = weight_e.iloc[1, :].to_frame().reset_index()
weight_e.columns = weight_e.iloc[0]
weight_e = weight_e[1:]
weight_e.rename(columns={"TIME": "date", "Germany": 'weight_e'}, inplace=True)
weight_e

/Users/lananhnguyen/anaconda3/envs/thesis/lib/python3.11/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,date,weight_e
1,2015,117.76
2,2016,103.55
3,2017,104.74
4,2018,103.74
5,2019,117.28
6,2020,108.7
7,2021,107.65
8,2022,120.65
9,2023,97.04
10,2024,104.27


In [13]:
weight_n = pd.read_excel('data/hicp_cat_raw/prc_hicp_weight_neig.xlsx', sheet_name='Sheet 1', skiprows=7)

weight_n = weight_n.iloc[1, :].to_frame().reset_index()
weight_n.columns = weight_n.iloc[0]
weight_n = weight_n[1:]
weight_n.rename(columns={"TIME": "date", "Germany": 'weight_n'}, inplace=True)
weight_n

/Users/lananhnguyen/anaconda3/envs/thesis/lib/python3.11/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,date,weight_n
1,2015,266.94
2,2016,269.89
3,2017,267.71
4,2018,267.04
5,2019,266.91
6,2020,265.55
7,2021,275.14
8,2022,266.84
9,2023,282.75
10,2024,279.86


In [14]:
weight_s = pd.read_excel('data/hicp_cat_raw/prc_hicp_weight_services.xlsx', sheet_name='Sheet 1', skiprows=7)

weight_s = weight_s.iloc[1, :].to_frame().reset_index()
weight_s.columns = weight_s.iloc[0]
weight_s = weight_s[1:]
weight_s.rename(columns={"TIME": "date", "Germany": 'weight_s'}, inplace=True)


/Users/lananhnguyen/anaconda3/envs/thesis/lib/python3.11/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [46]:
weights = pd.concat([weight_f, weight_e, weight_n, weight_s], axis=1)
weights = weights.loc[:, ~weights.columns.duplicated()]
weights['date'] = weights['date'].astype(int)
for col in weights.columns[1:]:
    weights[col] = weights[col].astype(float)

weights.loc[:, 'total'] = weights.iloc[:, 1:5].sum(axis=1)
weights


,date,weight_f,weight_e,weight_n,weight_s,total
1,2015,161.41,117.76,266.94,453.89,1000.0
2,2016,162.03,103.55,269.89,464.53,1000.0
3,2017,165.43,104.74,267.71,462.12,1000.0
4,2018,168.13,103.74,267.04,461.09,1000.0
5,2019,153.56,117.28,266.91,462.25,1000.0
6,2020,155.48,108.70,265.55,470.27,1000.0
7,2021,174.04,107.65,275.14,443.17,1000.0
8,2022,171.53,120.65,266.84,440.98,1000.0
9,2023,166.62,97.04,282.75,453.59,1000.0
10,2024,162.03,104.27,279.86,453.84,1000.0


In [33]:
weights

,date,weight_f,weight_e,weight_n,weight_s
1,2015,161.41,117.76,266.94,453.89
2,2016,162.03,103.55,269.89,464.53
3,2017,165.43,104.74,267.71,462.12
4,2018,168.13,103.74,267.04,461.09
5,2019,153.56,117.28,266.91,462.25
6,2020,155.48,108.7,265.55,470.27
7,2021,174.04,107.65,275.14,443.17
8,2022,171.53,120.65,266.84,440.98
9,2023,166.62,97.04,282.75,453.59
10,2024,162.03,104.27,279.86,453.84


In [16]:

food_unchain.name = 'food_unchain'
food_unchain = food_unchain.to_frame()
food_unchain.loc[:, 'year'] = food_unchain.index.year
food_unchain
merged_df = pd.merge(food_unchain, weights, how='inner', left_on='year', right_on='date')
mul_food = merged_df.loc[:, 'food_unchain'].mul(merged_df.loc[:, 'weight_f'])

In [17]:
energy_unchain.name = 'e_unchain'
e_unchain = energy_unchain.to_frame()
e_unchain.loc[:, 'year'] = e_unchain.index.year

merged_df = pd.merge(e_unchain, weights, how='inner', left_on='year', right_on='date')
mul_e = merged_df.loc[:, 'e_unchain'].mul(merged_df.loc[:, 'weight_e'])

In [18]:
neig_unchained.name = 'n_unchain'
n_unchain = neig_unchained.to_frame()
n_unchain.loc[:, 'year'] = n_unchain.index.year

merged_df = pd.merge(n_unchain, weights, how='inner', left_on='year', right_on='date')
mul_n = merged_df.loc[:, 'n_unchain'].mul(merged_df.loc[:, 'weight_n'])

In [19]:
services_unchained.name = 's_unchain'
s_unchain = services_unchained.to_frame()
s_unchain.loc[:, 'year'] = s_unchain.index.year

merged_df = pd.merge(s_unchain, weights, how='inner', left_on='year', right_on='date')
mul_s = merged_df.loc[:, 's_unchain'].mul(merged_df.loc[:, 'weight_s'])

In [47]:
mul_sum = pd.concat([mul_food, mul_e, mul_n, mul_s], axis=1)
mul_sum.loc[:, 'hicp_unchained'] = mul_sum.sum(axis=1)/1000

date_range = pd.date_range(start=mine_g.train_test_split_date + pd.DateOffset(months=1), end=mine_g.max_X_date, freq='M')
mul_sum.set_index(date_range, inplace=True)

In [51]:
mul_sum

,0,1,2,3,hicp_unchained
2016-01-31,162.845563,102.699374,266.713982,457.386127,0.989645
2016-02-29,162.757001,103.653113,269.050936,460.382534,0.995844
2016-03-31,162.933275,100.727021,272.00125,462.712822,0.998374
2016-04-30,164.131576,102.440934,271.954385,466.146823,1.004674
2016-05-31,164.424748,102.475382,272.130828,469.094546,1.008126
...,...,...,...,...,...
2022-08-31,189.68195,156.060534,277.038863,450.45003,1.073231
2022-09-30,192.141754,161.897365,280.637868,448.83016,1.083507
2022-10-31,195.112294,175.227818,282.521447,453.695811,1.106557
2022-11-30,197.690644,177.537925,285.931744,454.462623,1.115623


In [52]:
def transform_to_chained(y_unchain, y_real):
    y_real = y_real[y_real.index <= "2022-12-31"]
    dec_mask = y_real.index.month == 12
    dec_data = y_real.where(dec_mask, other=np.nan)
    dec_data.ffill(inplace=True)
    dec_data = dec_data[dec_data.index > "2015-12-31"]
    # chain data back:
    y_chained = y_unchain * dec_data
    return y_chained


y_real_hicp = mine_g.load_excel('data/hicp_all.xlsx', 'hicp_all', verbose=0)
hicp_chained_final = transform_to_chained(mul_sum.loc[:, 'hicp_unchained'], y_real_hicp.iloc[:, 0])

y_real_hicp.loc[:, 'last_y'] = y_real_hicp.iloc[:, 0].shift(12)
y_real_hicp = y_real_hicp[(y_real_hicp.index > mine_g.train_test_split_date) & (y_real_hicp.index <= mine_g.max_X_date)]

hicp_all_yoy = (hicp_chained_final/y_real_hicp.loc[:, 'last_y']-1)*100

In [53]:
hicp_all_yoy

2016-01-31     0.477723
2016-02-29       0.1879
2016-03-31    -0.262701
2016-04-30    -0.134229
2016-05-31    -0.189562
                ...    
2022-08-31     8.789301
2022-09-30     9.531649
2022-10-31    11.255497
2022-11-30    11.863813
2022-12-31    11.415892
Freq: M, Length: 84, dtype: object

In [54]:
hicp_all_yoy.name = 'pcr_h1'

In [55]:
forecast = pd.read_csv('data/headline_forecast/head_infl_forecast.csv')
new_forecast = pd.concat([forecast, hicp_all_yoy.reset_index(drop=True)], axis=1)
new_forecast

,ar_110_h_1,ar_110_h_2,ar_110_h_3,ar_ols_h_1,ar_ols_h_2,ar_ols_h_3,pcr_h1
0,0.201236,0.329779,1.160298,0.305073,0.400090,1.253522,0.477723
1,0.377993,0.201232,0.333175,0.493679,0.399921,0.486686,0.1879
2,-0.025772,0.382279,0.201233,0.029098,0.572149,0.486532,-0.262701
3,0.070035,-0.036898,0.381662,0.212929,0.147909,0.643806,-0.134229
4,-0.238173,0.074585,-0.035253,-0.152066,0.315777,0.256404,-0.189562
...,...,...,...,...,...,...,...
79,8.453284,8.295415,8.659386,7.862867,7.111033,6.988278,8.789301
80,8.796783,8.455887,8.295984,8.188482,7.301452,6.614902,9.531649
81,10.712508,8.800719,8.455609,10.074116,7.598793,6.788786,11.255497
82,11.523992,10.729384,8.800305,10.680094,9.320693,7.060308,11.863813


In [56]:
real_hicp_yoy = pd.read_csv('data/preprocessed/head_inflation.csv', parse_dates = True, index_col='date')
real_hicp_yoy_test = real_hicp_yoy[(real_hicp_yoy.index > mine_g.train_test_split_date) & (real_hicp_yoy.index <= mine_g.max_X_date)]

new_forecast = pd.concat([new_forecast, real_hicp_yoy_test.reset_index(drop=True)], axis=1)
new_forecast

,ar_110_h_1,ar_110_h_2,ar_110_h_3,ar_ols_h_1,ar_ols_h_2,ar_ols_h_3,pcr_h1,yoy_rate
0,0.201236,0.329779,1.160298,0.305073,0.400090,1.253522,0.477723,0.407747
1,0.377993,0.201232,0.333175,0.493679,0.399921,0.486686,0.1879,-0.101010
2,-0.025772,0.382279,0.201233,0.029098,0.572149,0.486532,-0.262701,0.100301
3,0.070035,-0.036898,0.381662,0.212929,0.147909,0.643806,-0.134229,-0.299401
4,-0.238173,0.074585,-0.035253,-0.152066,0.315777,0.256404,-0.189562,-0.099404
...,...,...,...,...,...,...,...,...
79,8.453284,8.295415,8.659386,7.862867,7.111033,6.988278,8.789301,8.834244
80,8.796783,8.455887,8.295984,8.188482,7.301452,6.614902,9.531649,10.899183
81,10.712508,8.800719,8.455609,10.074116,7.598793,6.788786,11.255497,11.562782
82,11.523992,10.729384,8.800305,10.680094,9.320693,7.060308,11.863813,11.261261


In [57]:
mean_absolute_error(new_forecast.loc[:, 'yoy_rate'], new_forecast.loc[:, 'pcr_h1'])

0.3824726170229611

In [60]:
np.sqrt(mean_squared_error(new_forecast.loc[:, 'yoy_rate'], new_forecast.loc[:, 'pcr_h1']))

0.5897982399466484

In [61]:
mean_absolute_error(new_forecast.loc[:, 'yoy_rate'], new_forecast.loc[:, 'ar_ols_h_1'])

0.46881208783747264

In [62]:
mean_absolute_error(new_forecast.loc[:, 'yoy_rate'], new_forecast.loc[:, 'ar_110_h_1'])

0.4334392868069618

# Compare with individual forecast:

In [3]:
food_forecast = pd.read_csv('data/forecast_results/food_forecast.csv')

real_food_yoy = pd.read_csv('data/preprocessed/food_yoy_infl.csv', parse_dates = True, index_col='date')
real_food_yoy_test = real_food_yoy[(real_food_yoy.index > mine_g.train_test_split_date) & (real_food_yoy.index <= mine_g.max_X_date)]

food_forecast = pd.concat([food_forecast, real_food_yoy_test.reset_index(drop=True)], axis=1)
food_forecast

,prc_food_h_1,prc_food_h_2,prc_food_h_3,yoy_rate
0,1.516479,2.246963,2.337054,1.209677
1,0.952438,1.290390,1.958365,1.103310
2,0.960510,0.849770,1.202369,1.402806
3,1.095242,0.736830,0.629217,1.195219
4,1.275819,1.282583,0.975238,0.896414
...,...,...,...,...
79,11.838994,9.102966,7.644693,12.849650
80,13.289327,12.178509,9.281201,14.423077
81,14.940325,13.680145,12.457030,15.458515
82,16.053798,15.364414,13.979912,16.275022


In [4]:
mean_absolute_error(food_forecast.loc[:, 'yoy_rate'], food_forecast.loc[:, 'prc_food_h_1'])

0.5009483647550693

In [15]:
np.sqrt(mean_squared_error(food_forecast.loc[:, 'yoy_rate'], food_forecast.loc[:, 'prc_food_h_1']))

0.7158016305008413

In [5]:
energy_forecast = pd.read_csv('data/forecast_results/energy_forecast.csv')

real_energy_yoy = pd.read_csv('data/preprocessed/energy_yoy_infl.csv', parse_dates = True, index_col='date')
real_energy_yoy_test = real_energy_yoy[(real_energy_yoy.index > mine_g.train_test_split_date) & (real_energy_yoy.index <= mine_g.max_X_date)]

energy_forecast = pd.concat([energy_forecast, real_energy_yoy_test.reset_index(drop=True)], axis=1)
energy_forecast

,prc_energy_h_1,prc_energy_h_2,prc_energy_h_3,yoy_rate
0,-4.150957,-3.265409,-5.095043,-5.696846
1,-5.284264,-3.636481,-2.915801,-8.565737
2,-9.223860,-5.986635,-4.513895,-8.939096
3,-8.130491,-8.671201,-5.410966,-8.504399
4,-8.989243,-8.679159,-8.868818,-8.034850
...,...,...,...,...
79,35.970997,39.937719,42.842723,35.831810
80,40.159641,39.399301,42.371777,44.232516
81,45.743418,40.834507,38.718992,43.542757
82,45.007611,47.702450,42.004431,40.102828


In [14]:
mean_absolute_error(energy_forecast.loc[:, 'yoy_rate'], energy_forecast.loc[:, 'prc_energy_h_1'])

2.093583667325028

In [13]:
np.sqrt(mean_squared_error(energy_forecast.loc[:, 'yoy_rate'], energy_forecast.loc[:, 'prc_energy_h_1']))


3.4889181466361996

In [17]:
neig_forecast = pd.read_csv('data/forecast_results/neig_forecast.csv')

real_neig_yoy = pd.read_csv('data/preprocessed/neig_yoy_infl.csv', parse_dates = True, index_col='date')
real_neig_yoy_test = real_neig_yoy[(real_neig_yoy.index > mine_g.train_test_split_date) & (real_neig_yoy.index <= mine_g.max_X_date)]

neig_forecast = pd.concat([neig_forecast, real_neig_yoy_test.reset_index(drop=True)], axis=1)
neig_forecast

,prc_neig_h_1,prc_neig_h_2,prc_neig_h_3,yoy_rate
0,0.729452,1.159671,1.253438,1.218274
1,1.201077,0.972342,1.229771,1.011122
2,0.983425,1.075789,0.900019,0.798403
3,0.764899,0.928747,1.017031,1.095618
4,1.031533,0.747216,0.858793,1.197605
...,...,...,...,...
79,5.869285,5.779228,5.249963,6.103286
80,6.148264,6.136310,5.697437,6.412639
81,6.464927,6.239703,6.149065,7.222222
82,7.352456,6.467392,6.336676,7.287823


In [18]:
mean_absolute_error(neig_forecast.loc[:, 'yoy_rate'], neig_forecast.loc[:, 'prc_neig_h_1'])

0.3505513158506578

In [19]:
np.sqrt(mean_squared_error(neig_forecast.loc[:, 'yoy_rate'], neig_forecast.loc[:, 'prc_neig_h_1']))

0.5666848861488682

In [20]:
s_forecast = pd.read_csv('data/forecast_results/services_forecast.csv')

real_s_yoy = pd.read_csv('data/preprocessed/services_yoy_infl.csv', parse_dates = True, index_col='date')
real_s_yoy_test = real_s_yoy[(real_s_yoy.index > mine_g.train_test_split_date) & (real_s_yoy.index <= mine_g.max_X_date)]

s_forecast = pd.concat([s_forecast, real_s_yoy_test.reset_index(drop=True)], axis=1)
s_forecast

,prc_services_h_1,prc_services_h_2,prc_services_h_3,yoy_rate
0,1.087786,1.241321,2.189706,1.128205
1,0.717032,0.635946,0.854914,0.913706
2,0.919456,0.818834,0.714219,1.619433
3,0.851811,0.617988,0.543636,0.401606
4,0.881835,1.091343,0.893010,0.798403
...,...,...,...,...
79,2.147497,2.427213,2.968636,1.722575
80,2.150609,2.274555,2.556510,3.639672
81,3.164126,2.753378,2.801649,3.909091
82,3.810354,3.414456,2.888919,3.835616


In [21]:
mean_absolute_error(s_forecast.loc[:, 'yoy_rate'], s_forecast.loc[:, 'prc_services_h_1'])

0.4631592214327238

In [22]:
np.sqrt(mean_squared_error(s_forecast.loc[:, 'yoy_rate'], s_forecast.loc[:, 'prc_services_h_1']))

0.5935097203570808